<a href="https://colab.research.google.com/github/wksmirnowa/machinelearning_homeworks/blob/master/HW01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Домашнее задание 1

Проанализируйте данные и найдите самый холодный и самый ветренный дни в Сегеде за всю имеющуюся историю.
Узнайте сколько раз шел дождь летом 2014 года.
Сгруппируйтей данные по годам, месяцам и неделям (по отдельности) и найдите:

1. самый холодный день в Сегеде (366-ой – 31.12)
2. самый ветренный день в Сегеде (5)
3. сколько раз шел дождь летом 2014 года
4. самый ветрянный месяц (март)
5. самый влажный год (2005)
6. неделю с самой переменчивой погодой (там, где отклонение какого-то или нескольких признаков самое большое) (6 неделя)



In [0]:
#!pip install pandas

In [0]:
import pandas as pd
import datetime

Подгружаем данные из kaggle

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


In [0]:
!kaggle datasets download -d budincsevity/szeged-weather -p /content/gdrive/My\ Drive/kaggle/szeged-weather

  0% 0.00/2.23M [00:00<?, ?B/s]
100% 2.23M/2.23M [00:00<00:00, 71.2MB/s]


In [0]:
!unzip '/content/gdrive/My Drive/kaggle/szeged-weather/szeged-weather.zip'

Archive:  /content/gdrive/My Drive/kaggle/szeged-weather/szeged-weather.zip
  inflating: weatherHistory.csv      


In [0]:
import os
os.chdir('/content/gdrive/My Drive/kaggle/szeged-weather')

In [0]:
!unzip -q szeged-weather.zip

Читаем и редактируем данные из таблички

In [0]:
weather_hist = pd.read_csv('weatherHistory.csv')

In [0]:
weather_hist.rename({'Formatted Date':'date', 'Temperature (C)':'temp', 'Apparent Temperature (C)':'temp_app',
                    'Precip Type':'weather_type', 'Wind Speed (km/h)':'wind_speed', 'Humidity':'humidity',
                    'Daily Summary':'summary'}, 
                    axis=1, inplace=True)

In [0]:
weather_hist.drop(['Loud Cover'], axis=1, inplace=True)

Группируем данные по дате и меняем тип даты на Datetime

In [0]:
weather_hist['date'] = pd.to_datetime(weather_hist['date'], utc=True)

In [0]:
weather_hist.dtypes #проверяем

date                      datetime64[ns, UTC]
Summary                                object
weather_type                           object
temp                                  float64
temp_app                              float64
humidity                              float64
wind_speed                            float64
Wind Bearing (degrees)                float64
Visibility (km)                       float64
Pressure (millibars)                  float64
summary                                object
dtype: object

### Самый влажный год

Найдем самый влажный год. Для этого сгруппируем данные по годам и средним значениям.

In [0]:
weather_hist['year'] = weather_hist['date'].dt.year
group_by_year = weather_hist.groupby('year').mean()

In [0]:
group_by_year

,temp,temp_app,humidity,wind_speed,Wind Bearing (degrees),Visibility (km),Pressure (millibars)
year,,,,,,,
2005,0.577778,-4.050000,0.890000,17.114300,140.000000,9.982000,1016.660000
2006,11.215225,10.158189,0.767349,10.189128,189.339726,9.757280,992.545486
2007,12.134677,11.062986,0.689647,10.824162,194.106393,10.408192,1001.639412
2008,12.161819,11.053128,0.701239,11.304172,193.860997,10.266882,1007.734488
2009,12.269682,11.070866,0.707247,11.507326,180.477740,10.002187,1002.605055
2010,11.200176,10.097002,0.796861,11.014343,192.078780,9.140282,1004.815439
2011,11.524934,10.584490,0.736008,9.898596,189.334475,9.508237,1014.183506
2012,11.986824,10.753117,0.689513,11.264745,187.746926,10.362670,999.341404
2013,11.941017,10.825509,0.754197,10.970036,187.862557,10.892672,1004.951297


In [0]:
most_humidity_year = group_by_year.loc[group_by_year['humidity'] == group_by_year['humidity'].max()]

In [0]:
most_humidity_year.index #самый влажный год 2005

Int64Index([2005], dtype='int64', name='year')

### Самый ветреный месяц


In [0]:
weather_hist['month'] = weather_hist['date'].dt.month
group_by_month = weather_hist.groupby('month').mean()

In [0]:
group_by_month

,temp,temp_app,humidity,wind_speed,Wind Bearing (degrees),Visibility (km),Pressure (millibars),year,day,dayofyear,week
month,,,,,,,,,,,
1,0.815678,-1.938761,0.850698,11.515865,180.460655,7.835825,1006.245222,2011.000000,16.000000,16.000000,4.255132
2,2.166880,-0.555908,0.813407,12.180815,188.733253,8.730536,1003.925126,2011.009646,45.639871,45.639871,6.948553
3,6.914976,5.082583,0.702849,13.415825,186.632698,10.922231,1001.564999,2011.000000,75.272727,75.272727,11.181818
4,12.769200,12.158287,0.641051,11.876139,186.435859,11.784991,1009.995154,2011.000000,105.772727,105.772727,15.539394
5,16.874455,16.733265,0.691650,10.953186,188.143328,11.889730,1003.376970,2011.000000,136.272727,136.272727,19.894428
6,20.728510,20.755415,0.686418,9.632453,202.804924,11.994000,1002.019582,2011.000000,166.772727,166.772727,24.254545
7,22.965462,22.977605,0.639593,9.637124,202.937049,12.188605,1008.562854,2010.998900,197.273805,197.273805,28.610072
8,22.339134,22.264600,0.635300,8.942707,195.211135,12.454307,1001.470591,2010.997076,228.230750,228.230750,33.031920
9,17.503458,17.389856,0.689163,9.610204,187.515025,11.598414,1000.835433,2011.000000,258.772727,258.772727,37.396970


In [0]:
group_by_month_info = group_by_month.loc[group_by_month['wind_speed'] == group_by_month['wind_speed'].max()]
most_windy_month = group_by_month_info.index

In [0]:
most_windy_month #Самый ветреный месяц – март (3)

Int64Index([3], dtype='int64', name='month')

### Самый холодный день

In [0]:
weather_hist['dayofyear'] = weather_hist['date'].dt.dayofyear
group_by_day = weather_hist.groupby('dayofyear').mean()

In [0]:
group_by_day

,temp,temp_app,humidity,wind_speed,Wind Bearing (degrees),Visibility (km),Pressure (millibars),year,month,day
dayofyear,,,,,,,,,,
1,-1.115867,-3.870328,0.862121,9.990172,175.746212,6.737362,966.192614,2011.000000,1.0,1.0
2,-0.329861,-3.148548,0.871023,10.961417,204.556818,6.492691,957.603333,2011.000000,1.0,2.0
3,-0.459680,-3.098506,0.860189,10.185628,209.162879,8.015239,965.126326,2011.000000,1.0,3.0
4,-0.689352,-4.051557,0.839242,13.498752,170.761364,8.591545,951.901288,2011.000000,1.0,4.0
5,0.455324,-3.002567,0.840682,14.045664,178.053030,8.240822,966.847121,2011.000000,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...
362,-0.677715,-3.036111,0.899735,9.576390,167.034091,5.228719,978.635833,2011.000000,12.0,362.0
363,-0.794949,-3.342003,0.872803,9.597552,191.753788,5.890039,963.357083,2011.000000,12.0,363.0
364,-1.610375,-4.338678,0.856932,9.417463,149.363636,6.254850,960.693788,2011.000000,12.0,364.0


In [0]:
coldest_day_info = group_by_day.loc[group_by_day['temp'] == group_by_day['temp'].min()]
coldest_day_info.index

Int64Index([366], dtype='int64', name='dayofyear')

In [0]:
# можно посчитать просто день с самой минимальной температурой, но мне кажется, что это
# не самое верное решение и что нужно смотреть день, у которого в среднем самая низкая температура
# weather_hist['day'] = weather_hist['date'].dt.day
# group_by_day_d = weather_hist.groupby('day').min()
# coldest_day_d_info = group_by_day_d.loc[group_by_day_d['temp'] == group_by_day_d['temp'].min()]
# coldest_day_d_info

### Самый ветреный день

In [0]:
fastest_day = group_by_day.loc[group_by_day['wind_speed'] == group_by_day['wind_speed'].max()]
fastest_day.index

Int64Index([5], dtype='int64', name='dayofyear')

Если нужно будет посмотреть точные даты, можно сгруппировать данные сразу по 3 признакам, но, к сожалению, тогда появляются проблемы с вытаскиванием индекса.

In [0]:
weather_hist['year'] = weather_hist['date'].dt.year
weather_hist['month'] = weather_hist['date'].dt.month
weather_hist['day'] = weather_hist['date'].dt.day
group_by_day = weather_hist.groupby(['year','month','day']).agg(['mean'])
group_by_date

temp  temp_app  humidity  ...  hour minute second
                    mean      mean      mean  ...  mean   mean   mean
year month day                                ...                    
2005 12    31   0.577778 -4.050000  0.890000  ...  23.0      0      0
2006 1     1    4.075000 -0.174537  0.817083  ...  11.5      0      0
           2    5.263194  1.808796  0.847083  ...  11.5      0      0
           3    2.340509  0.445370  0.897083  ...  11.5      0      0
           4    2.251852 -0.663194  0.906667  ...  11.5      0      0
...                  ...       ...       ...  ...   ...    ...    ...
2016 12    27   0.278009 -3.160417  0.890000  ...  11.5      0      0
           28   0.222222 -3.225926  0.890000  ...  11.5      0      0
           29   0.167824 -3.265509  0.889583  ...  11.5      0      0
           30   0.117593 -3.309259  0.889167  ...  11.5      0      0
           31   0.113527 -3.320773  0.886522  ...  11.0      0      0

[4019 rows x 11 columns]

### Неделя с самой переменчивой погодой

In [0]:
weather_hist['week'] = weather_hist['date'].dt.week
group_by_week = weather_hist.groupby('week').std()

In [0]:
group_by_week

,temp,temp_app,humidity,wind_speed,Wind Bearing (degrees),Visibility (km),Pressure (millibars),year,month,day,dayofyear
week,,,,,,,,,,,
1,5.008042,5.530550,0.101536,7.393594,97.107004,4.314909,246.685649,3.135699,3.698766,121.071078,121.071078
2,4.811115,5.394467,0.113069,6.531584,99.837951,4.991813,89.103028,3.163134,0.000000,2.892778,2.892778
3,4.532339,5.165160,0.107841,6.884513,91.283170,4.638049,67.561625,3.163134,0.000000,2.892778,2.892778
4,5.482606,6.102784,0.118193,7.777317,111.060259,4.480531,42.686836,3.163134,0.000000,2.892778,2.892778
5,5.251543,5.925691,0.132209,8.692668,104.658320,4.409161,122.760869,3.163134,0.495006,2.892778,2.892778
6,6.456311,7.246163,0.129590,7.094790,99.705722,4.623077,110.584848,3.163134,0.000000,2.892778,2.892778
7,4.628565,5.535001,0.127673,7.037540,99.560439,4.267868,160.344471,3.163134,0.000000,2.892778,2.892778
8,4.967552,5.908516,0.148057,7.138245,105.554739,4.156512,48.021065,3.163134,0.000000,2.892778,2.892778
9,4.839419,5.658514,0.153718,7.207053,98.208495,3.995919,105.512072,3.163134,0.499080,2.892778,2.892778


In [0]:
crazy_week = group_by_week.loc[group_by_week['temp'] == group_by_week['temp'].max()]
crazy_week.index

Int64Index([6], dtype='int64', name='week')

### Сколько раз шел дождь летом 2014

Можно пойти простым путем: просто посчитать, сколько строк с типом погоды 'rain' есть в данных. К сожалению, с нашими данными так мы не сможем узнать точное число дождливых дней. Для этого нам нужно перейти к категориальным переменным, с подсчетом которых возникают свои трудности.


In [0]:
summer = weather_hist.loc[weather_hist['weather_type'] == 'rain']
#можно пойти этим путем и посчитать, сколько строк с типом погоды 'rain', но так
#мы не сможем узнать точное число дождливых дней, нам нужно перейти к 
#категориальным переменным

In [0]:
summer.count

In [0]:
weather_hist['year'] = weather_hist['date'].dt.year
weather_hist['month'] = weather_hist['date'].dt.month
weather_hist['day'] = weather_hist['date'].dt.day
group_by_date1 = weather_hist.groupby('year').count()

In [0]:
group_by_date1

,date,Summary,weather_type,temp,temp_app,humidity,wind_speed,Wind Bearing (degrees),Visibility (km),Pressure (millibars),summary,month,day,dayofyear,week
year,,,,,,,,,,,,,,,
2005,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2006,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760
2007,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760
2008,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784
2009,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760
2010,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784
2011,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760
2012,8784,8784,8641,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784,8784
2013,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760


In [0]:
# year2014 = group_by_date1.loc[group_by_date1['year'] == group_by_date1['2014']]
# year2014

## Домашнее задание 2

Придумать признаки (как минимум 5), которые можно извлечь из текста, помимо тех 4, которые мы уже использовали. 

Обучите модель на них и проверьте, улучашается ли качество. 

Опишите признаки, которое придумали и почему считаете, что они должны показывать вероятность покупки.

Побробуйте подобрать параметры векторизатора (TfidfVectorizer или CountVectorizer, или обоих) и параметры алгоритма так, чтобы качество, которое у нас получилось (0.248) улучшилось. 

Опишите то, что попробовали.


In [0]:
import pandas as pd

In [0]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import LinearSVR
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack
from sklearn.metrics import r2_score
from sklearn.linear_model import SGDRegressor
import string


In [0]:
import os
os.chdir('/content/gdrive/')

In [0]:
data = pd.read_csv('data.csv')
data.dropna(subset=['description'], inplace=True)
data = data.sample(frac=0.08)
data.to_csv('data.csv', index=None)
data = pd.read_csv('data.csv')

In [0]:
sorted_data = data.sort_values('deal_probability', ascending=False) #посмотрим на данные

In [0]:
sorted_data[:10]

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_noncaps,number_of_nums,number_of_spaces,number_of_punct,words
8719,d486827679d6,068fca56e063,Ханты-Мансийский АО,Сургут,Услуги,Предложение услуг,"Реклама, полиграфия","Полиграфия, дизайн",NaN,Печать визиток,"Печать визиток от 0,552руб/шт /\nПечать листов...",1.0,216,2017-03-22,Company,0ee870e2c5be5b051eaec70859f6b29fe0a73d22fc1f04...,689.0,1.0,131,22,5,29,88,9,21,131,0
9747,fb21cf55769f,60bcc306dc65,Нижегородская область,Нижний Новгород,Услуги,Предложение услуг,"Обучение, курсы",Профессиональная подготовка,NaN,Курсы сварщиков,Форма обучения: Вечерняя/\nО программе./\nОбуч...,7500.0,1,2017-03-28,Private,13596cbd6187bfe9dbfff5f6e7fa9da3750f9c97088675...,2266.0,1.0,868,99,12,126,730,0,98,868,0
8090,57916d9f508c,816a0e58be0b,Белгородская область,Белгород,Услуги,Предложение услуг,"Ремонт, строительство","Строительство домов, коттеджей",NaN,Забор и навес,Заборы и навесы. Звоните договоримся. Цена от ...,300.0,63,2017-03-21,Company,15bdf0b5b874bf90745388ecad2db6803f0f555802eb52...,1284.0,1.0,63,11,4,14,42,3,10,63,0
4808,7aec22aca86f,0810319602ff,Тульская область,Тула,Услуги,Предложение услуг,Уборка,Генеральная уборка,NaN,"Уборка квартиры,дома,офиса",Качественно и не дорого окажу услугу по уборке...,1000.0,52,2017-03-27,Private,745f86509048f7287387e9849ac26a4b4ce8faa3c78efa...,1404.0,1.0,159,18,7,26,126,0,17,159,0
611,dee8c64f9865,52cacb0c9cc0,Пермский край,Пермь,Услуги,Предложение услуг,"Транспорт, перевозки",Аренда авто,NaN,Сдам в аренду,Авто в полной комплектации аренда 800 руб/суто...,800.0,1,2017-03-21,Private,52a22912d9734a269334b03fab2c4eb6f9d229d9a58926...,1113.0,1.0,106,13,1,16,82,7,12,106,0
857,5c8080f58fcc,b8209bcae085,Воронежская область,Калач,Услуги,Предложение услуг,"Ремонт, строительство",Отделочные работы,NaN,"Натяжные потолки, окна, балконы, кровля, стройка","Натяжные потолки в Калаче, Воронеже, и чернозе...",295.0,17,2017-03-18,Company,b365ab98ef152fbadf425c9e0bb86e08a5023796b0e291...,1191.0,1.0,760,90,27,149,577,7,95,760,0
3965,3894474d3cfa,cf938759bf23,Ханты-Мансийский АО,Нижневартовск,Услуги,Предложение услуг,Другое,NaN,NaN,"Курсовые, дипломные","Помогу в написании курсовых, дипломных работ в...",1.0,2,2017-03-16,Private,NaN,NaN,1.0,70,10,1,12,57,0,9,70,0
4556,bf63f3c50ee9,7fd1447f1690,Пермский край,Пермь,Услуги,Предложение услуг,"Транспорт, перевозки",Аренда авто,NaN,Логан. Аренда-выкуп.Под такси,Декабрь 2016 г.в.Белый.Без пробега.Пропан.Цент...,1500.0,71,2017-03-28,Company,NaN,NaN,1.0,272,28,24,47,183,18,27,272,0
2491,a9f8e0995349,2ca104d4276f,Ханты-Мансийский АО,Сургут,Услуги,Предложение услуг,Деловые услуги,"Бухгалтерия, финансы",NaN,Декларация 3 ндфл,Быстро и качественно заполню декларацию 3 НДФЛ...,300.0,6,2017-03-25,Private,a6978586a943e6c9097b3f35af3628dcbc4117e0c3a680...,2890.0,1.0,182,25,6,32,140,4,24,182,1
8305,f9723ec81c0f,a41036293337,Ростовская область,Ростов-на-Дону,Услуги,Предложение услуг,Уборка,Генеральная уборка,NaN,Клининг,"Уборка (в т..ч. генеральная) квартир, домов, о...",NaN,109,2017-03-23,Private,61ec9aaef4f63e4c735ecbc180e7041f7c1f0d85ae32f7...,2272.0,1.0,164,19,5,35,124,0,19,164,0


Изначально в качестве дополнительных параметров я брала:

* число маленьких букв

Моя логика такова: в тексте объявления, по которому будут что-то покупать, не должны быть только маленькие буквы, но их должно быть больше, чем больших. То есть вероятность того, что купят что-то по объявлению, больше у того объявления, у которого количество больших и маленьких букв сбалансированно в пользу большего количества маленьких букв.

* число цифр

Думаю, что использование числовых признаков/параметров товаров и услуг может повлиять на вероятность покупки в сторону увеличения.

* число пробелов

Если в тексте мало пробелов, это должно отразиться на продаваемости, так как текст будет менее читабельный.

* пунктуация

Знаки препинания должны быть в более продаваемом тексте, но в умеренном количестве.

* специальные слова

Возможно, слова "внимание", "срочно" и т.д. повлияют на вероятность покупки товара/услуги.

Если выбрать какой-то один дополнительный параметр, то можно улучшить модель, не сломав при этом изначальную метрику.

In [0]:
data['text_len'] = data['description'].apply(len)
data['text_len_tokens'] = data['description'].apply(lambda x: len(x.split()))
data['number_of_caps'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isupper()]))
data['number_of_nonalphanum'] = data['description'].apply(lambda x: len([ch for ch in x if not ch.isalnum()]))
data['number_of_noncaps'] = data['description'].apply(lambda x: len([ch for ch in x if ch.islower()]))
# data['number_of_nums'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isdigit()]))
# data['number_of_spaces'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isspace()]))
# selling_words = ('внимание', 'срочно', 'недорого', 'быстро', 'дешево', 'скидка') 
# data['words'] = data['description'].apply(lambda x: len([w for w in x.lower().split() if w in selling_words]))
# data['number_of_spaces'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isspace()]))
# data['number_of_punct'] = data['description'].apply(lambda x: len([ch for ch in x if ch in string.punctuation]))

In [0]:
X = data[['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_noncaps']].values
y = data.deal_probability.values

In [0]:
X.shape

(11097, 7)

In [0]:
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2))
tfidf = TfidfVectorizer(max_features=1000, min_df=5,analyzer='char', ngram_range=(3,3), max_df=0.4)
y = data.deal_probability.values
X_cv = cv.fit_transform(data.description)
X_tfidf = tfidf.fit_transform(data.description)

X = hstack([X_cv, X_tfidf])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
regressor = SGDRegressor(max_iter=50, alpha=0.00001)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds)) #должно быть лучше 0.248

0.2673904377705585

In [0]:
regressor = Ridge(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))  #должно быть лучше 0.248

In [0]:
r2_score(y_test, preds)

0.0416272004751459

Но как только я добавляю все оставшиеся параметры, происходит вот это.

In [0]:
data['text_len'] = data['description'].apply(len)
data['text_len_tokens'] = data['description'].apply(lambda x: len(x.split()))
data['number_of_caps'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isupper()]))
data['number_of_nonalphanum'] = data['description'].apply(lambda x: len([ch for ch in x if not ch.isalnum()]))
data['number_of_noncaps'] = data['description'].apply(lambda x: len([ch for ch in x if ch.islower()]))
data['number_of_nums'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isdigit()]))
data['number_of_spaces'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isspace()]))
selling_words = ('внимание', 'срочно', 'недорого', 'быстро', 'дешево', 'скидка') 
data['words'] = data['description'].apply(lambda x: len([w for w in x.lower().split() if w in selling_words]))
data['number_of_punct'] = data['description'].apply(lambda x: len([ch for ch in x if ch in string.punctuation]))
X = data[['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_noncaps', 'number_of_nums', 'number_of_spaces', 'words', 'number_of_punct']].values
y = data.deal_probability.values
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2))
tfidf = TfidfVectorizer(max_features=1000, min_df=5,analyzer='char', ngram_range=(3,3), max_df=0.4)
y = data.deal_probability.values
X_cv = cv.fit_transform(data.description)
X_tfidf = tfidf.fit_transform(data.description)

X = hstack([X_cv, X_tfidf])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
regressor = SGDRegressor(max_iter=50, alpha=0.00001)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds)) #должно быть лучше 0.248

0.45752954259566464

In [0]:
regressor = Ridge(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))  #должно быть лучше 0.248

0.2807670339275486

In [0]:
r2_score(y_test, preds)

-0.05665925844742703

То есть частично что-то ломается, хотя SGDRegressor увеличился почти вдвое. Попробуем поменять параметры векторайзера.

In [0]:
data['text_len'] = data['description'].apply(len)
data['text_len_tokens'] = data['description'].apply(lambda x: len(x.split()))
data['number_of_caps'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isupper()]))
data['number_of_nonalphanum'] = data['description'].apply(lambda x: len([ch for ch in x if not ch.isalnum()]))
data['number_of_noncaps'] = data['description'].apply(lambda x: len([ch for ch in x if ch.islower()]))
data['number_of_nums'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isdigit()]))
data['number_of_spaces'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isspace()]))
selling_words = ('внимание', 'срочно', 'недорого', 'быстро', 'дешево', 'скидка') 
data['words'] = data['description'].apply(lambda x: len([w for w in x.lower().split() if w in selling_words]))
data['number_of_punct'] = data['description'].apply(lambda x: len([ch for ch in x if ch in string.punctuation]))
X = data[['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_noncaps', 'number_of_nums', 'number_of_spaces', 'words', 'number_of_punct']].values
y = data.deal_probability.values
cv = CountVectorizer(max_features=10000, min_df=4, max_df=0.4, ngram_range=(3,3))
tfidf = TfidfVectorizer(max_features=10000, min_df=4,analyzer='char', ngram_range=(3,3), max_df=0.4)
y = data.deal_probability.values
X_cv = cv.fit_transform(data.description)
X_tfidf = tfidf.fit_transform(data.description)

X = hstack([X_cv, X_tfidf])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
regressor = SGDRegressor(max_iter=50, alpha=0.00001)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds)) #должно быть лучше 0.248

0.2700575141779264

In [0]:
regressor = Ridge(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))  #должно быть лучше 0.248

0.27784457193950585

In [0]:
r2_score(y_test, preds)

0.022413341245524965

Вот так значение метрики стало гораздо лучше.